In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import flatten

In [43]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block2 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block3 = nn.Sequential(
        nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block4 = nn.Sequential(
        nn.Conv2d(256, 512, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block5 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Sequential(
        nn.Linear(512*1*1, 4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(4096, 100)
        )


    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [44]:
model = VGG16().to(device)

In [45]:
model

VGG16(
  (block1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8)

In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

In [22]:
train_dataset = datasets.CIFAR100(root='data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

100%|██████████| 169M/169M [00:13<00:00, 13.0MB/s]


Extracting data/cifar-100-python.tar.gz to data


In [47]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {epoch_loss:.4f}')

print('Training complete')

Epoch [1/10], Average Loss: 4.6072
Epoch [2/10], Average Loss: 4.6060
Epoch [3/10], Average Loss: 4.6060
Epoch [4/10], Average Loss: 4.6059
Epoch [5/10], Average Loss: 4.6060
Epoch [6/10], Average Loss: 4.6059
Epoch [7/10], Average Loss: 4.6059
Epoch [8/10], Average Loss: 4.6059
Epoch [9/10], Average Loss: 4.6059
Epoch [10/10], Average Loss: 4.6059
Training complete
